In [14]:
import io
import re
import pickle
import numpy as np
import networkx as nx
import operator
from collections import Counter
import nltk

In [2]:
# Get all the episodes' names in season.txt
# It is like getting the total philosophers' names
f1 = io.open('season.txt', 'r', encoding='utf-8')
text = f1.read()
episodesname = re.findall(r'\[\[(.*?)[\|\]\]]', text)
f1.close()

In [3]:
# Load the LabMT file which is the file containing the sentiment words and their score
f1 = io.open('LabMT.txt', 'r' ,encoding='utf-8')
wlist=f1.read().split()
wordlist=wlist[15:len(wlist)]

In [4]:
def sentiment(textfile):
    if not isinstance(textfile,list):
        #Load the textfile that should be analyzed
        text1 = io.open(textfile, 'r',encoding='utf-8')
        textlist=text1.read().split()
        text1.close()
    else:
        textlist=textfile
    #Remove all numbers.
    nonumbers=re.compile("[^\d]+").findall(' '.join(textlist))
    #Remove everything that is not words and return their lowercase.
    onlywords=re.compile("[\w|']+").findall(' '.join(nonumbers))
    finallist=[w.lower() for w in onlywords]

    #Make a list of all the words in the textfile and their count
    counts = Counter(finallist)
    most_common_list=counts.most_common()

    #Set the score to 0 and the word count to 0.
    score=0
    wordcount=0
    for i in range(len(most_common_list)):
        #Get a word in the textfile that is being scored.
        curword=most_common_list[i]
        #If the list in the text is part of the sentiment wordlist do the following.
        if curword[0] in wordlist:
            #Find the index of the word in the wordlist
            #and then add the score times the number of times the words appear to score
            wordindex=wordlist.index(curword[0])
            #The score is always 2 places further down in the list compared to the word.
            if wordlist[wordindex+2]!='--':
                score+=float(wordlist[wordindex+2])*curword[1]
                wordcount+=curword[1]
    if score==0:
        if not isinstance(textfile,list):
            print "None of the words in %s was in the LabMT wordlist." %textfile
        else:
            print "None of the words in the text file was in the LabMT wordlist."
    else:
        return score/wordcount

## Character sentiment score
* Count the frequency of the characters in each episodes

* If the character appears more than x times, calculate the sentiment score and give the score to the character

* Store the score of each character and average them


In [5]:
-

-----season 1-----
-----season 2-----
-----season 3-----
-----season 4-----
-----season 5-----
-----season 6-----
done


In [6]:
charkeys=sentdict.keys()
chardictmean={}
for name in charkeys:
    chardictmean[name]=np.mean(sentdict[name])
    
char_sent_sorted=sorted(chardictmean.items(), key=operator.itemgetter(1),reverse=True)
char_sent_sorted[:10]

[('Jhiqui', 5.3829896259037984),
 ('Wylla', 5.3829896259037984),
 ('Catspaw assassin', 5.3829896259037984),
 ('Lady', 5.3829896259037984),
 ('Lothar Frey', 5.3748657821660046),
 ('Prendahl na Ghezn', 5.3706265457543241),
 ('Mero', 5.3706265457543241),
 ('Pentoshi servant', 5.3638706256627726),
 ('Waymar Royce', 5.3638706256627726),
 ('Wildling girl', 5.3638706256627726)]

* After we've done this, consider their sentiment score with attributes like their houses, dead/alive. See if a dead person has lower sentiment score, and/or compare the sentiment average score of different houses

In [7]:
#Load directed graph, undirected graph and the list of philosophers.
dg = nx.read_gpickle("Digraph_GOT_with_attributes.gpickle")

#Generate the largest connected component.
subdg = list(nx.weakly_connected_component_subgraphs(dg))
dg=subdg[0]


In [8]:
dead=[]
alive=[]

for name in sentdict.keys():
    if nx.get_node_attributes(dg,"Status")[name]=="Dead":
        dead.append(np.mean(sentdict[name]))
    if nx.get_node_attributes(dg,"Status")[name]=="Alive":
        alive.append(np.mean(sentdict[name]))

print "Mean sentiment of dead characters: %f" % np.mean(dead)
print "Mean sentiment of Alive characters: %f" % np.mean(alive)

Mean sentiment of dead characters: 5.315106
Mean sentiment of Alive characters: 5.313668


In [9]:
allhouses=nx.get_node_attributes(dg,"House").values()
unique_houses = list(set(x for l in allhouses for x in l))
housesentdict={}

for name in sentdict.keys():
    for house in nx.get_node_attributes(dg,"House")[name]:
        if house not in housesentdict.keys():
            housesentdict[house]=[]
        housesentdict[house].append(np.mean(sentdict[name]))


In [10]:
housekeys=housesentdict.keys()
housedictmean={}
for name in housekeys:
    housedictmean[name]=np.mean(housesentdict[name])
    
house_sent_sorted=sorted(housedictmean.items(), key=operator.itemgetter(1),reverse=True)
house_sent_sorted[:10]

[(u'Sansa Stark', 5.3829896259037984),
 (u'Illyrio Mopatis', 5.3638706256627726),
 (u'Dragonstone', 5.3602229617304502),
 (u'House Brune', 5.3602229617304502),
 (u'Second Sons (mercenary company)', 5.3580010187065659),
 (u'Xaro Xhoan Daxos', 5.353008964304145),
 (u'Craster', 5.348124481327801),
 (u'House Egen', 5.3415612595924786),
 (u'Khal', 5.3414816817853108),
 (u'House Payne', 5.3413611113208548)]